# Welcome to my LSTM Model! Hit run and watch the step-by-step training of my stock prediction Model

At the end you will get a LSTM_model file which can be plugged into a different script @/stockpredictionapp.ipynb which can be applied to all stocks and not just the 9 tested here!

In [10]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

# Define the tickers to fetch data for
tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'NVDA', 'ADBE', 'INTC', 'CSCO', 'IBM']

# Fetch data for each ticker using Yahoo Finance API
data = {}
for ticker in tickers:
    stock = yf.Ticker(ticker)
    data[ticker] = stock.history(period='5y')['Close']

# Combine data for all stocks into a single dataset
df = pd.concat(data.values(), axis=1, keys=data.keys())

df

,AAPL,GOOGL,MSFT,TSLA,NVDA,ADBE,INTC,CSCO,IBM
Date,,,,,,,,,
2018-04-16 00:00:00-04:00,41.903233,52.305000,88.875267,19.414000,57.314465,226.520004,45.375809,37.216469,118.488937
2018-04-17 00:00:00-04:00,42.479984,53.967999,90.668427,19.179333,58.812378,231.160004,46.362984,38.325241,120.755363
2018-04-18 00:00:00-04:00,42.384659,53.769501,91.017639,19.556667,58.522705,231.100006,46.423595,38.359608,111.659859
2018-04-19 00:00:00-04:00,41.183472,54.472500,90.706200,20.005333,56.707878,227.619995,45.219933,38.299442,110.841858
2018-04-20 00:00:00-04:00,39.496094,53.866001,89.658592,19.349333,56.626175,225.240005,44.622425,37.895477,108.740608
...,...,...,...,...,...,...,...,...,...
2023-04-10 00:00:00-04:00,162.029999,106.440002,289.390015,184.509995,275.790009,376.250000,32.520000,51.330002,131.029999
2023-04-11 00:00:00-04:00,160.800003,105.349998,282.829987,186.789993,271.690002,371.109985,32.349998,50.619999,130.419998
2023-04-12 00:00:00-04:00,160.100006,104.639999,283.489990,180.539993,264.950012,369.890015,32.020000,50.110001,128.539993


# This data will now be shaped so it can be inputted into the machine learning model!

In [2]:
# scale the data
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(df.values.reshape(-1,1))

# split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]


In [3]:
# create input/output arrays for training and testing data
def create_dataset(data, lookback):
    X, Y = [], []
    for i in range(lookback, len(data)):
        X.append(data[i-lookback:i, 0])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

lookback = 60
x_train, y_train = create_dataset(train_data, lookback)
x_test, y_test = create_dataset(test_data, lookback)


# Time to build the model!
1. creating the models
2. compliing the model
3. fitting the data into the model

In [4]:

# Build Model
model = Sequential() 
model.add(LSTM(50, return_sequences=True, input_shape=(lookback, 1)))
model.add(LSTM(50, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))

# compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# train the model
model.fit(x_train, y_train, batch_size=1, epochs=1)


9004/9004 [==============================] - 887s 98ms/step - loss: 0.0106


# Heres what our model looks like so far!

In [16]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 50)            10400     
                                                                 
 lstm_1 (LSTM)               (None, 50)                20200     
                                                                 
 dense (Dense)               (None, 25)                1275      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 31,901
Trainable params: 31,901
Non-trainable params: 0
_________________________________________________________________
None


# Now that the model is created, we can start to make our predictions!


In [17]:
# make predictions and evaluate performance
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)
mae = mean_absolute_error(y_test, predictions)
print('Mean Absolute Error:', mae)

# pickle data
model.save("lstm_model")


Mean Absolute Error: 143.95195703801957


In [ ]:
plt.plot(y_test, label=True)
plt.plot(predictions, label="LSTM Value")
plt.plot(predicted_price, label="Predicted Price Value")
plt.title("Prediction by LSTM")
plt.xlabel("Time Scale")
plt.ylabel("USD")
plt.legend()
plt.show()